<a href="https://colab.research.google.com/github/ricardobizerra/data-analysis/blob/main/predicting_titanic's_survivors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import pandas as pd
import numpy as np
import torch

In [126]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [127]:
# columns that are not relevant for the analysis are removed
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis='columns', inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [128]:
df.size

7128

In [129]:
# people that were 35
df[df['Age'] == 35]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
20,0,2,male,35.0,0,0,26.0000,S
211,1,2,female,35.0,0,0,21.0000,S
230,1,1,female,35.0,1,0,83.4750,S
258,1,1,female,35.0,0,0,512.3292,C
269,1,1,female,35.0,0,0,135.6333,S
279,1,3,female,35.0,1,1,20.2500,S
363,0,3,male,35.0,0,0,7.0500,S
383,1,1,female,35.0,1,0,52.0000,S


In [130]:
# removing empty or NaN data
df.dropna(inplace=True)
df.size

5696

In [131]:
df['AgeNorm'] = np.log10(df['Age'])
df['FareNorm'] = np.log10(df['Fare'])
df

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeNorm,FareNorm
0,0,3,male,22.0,1,0,7.2500,S,1.342423,0.860338
1,1,1,female,38.0,1,0,71.2833,C,1.579784,1.852988
2,1,3,female,26.0,0,0,7.9250,S,1.414973,0.898999
3,1,1,female,35.0,1,0,53.1000,S,1.544068,1.725095
4,0,3,male,35.0,0,0,8.0500,S,1.544068,0.905796
...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,1.591065,1.464266
886,0,2,male,27.0,0,0,13.0000,S,1.431364,1.113943
887,1,1,female,19.0,0,0,30.0000,S,1.278754,1.477121
889,1,1,male,26.0,0,0,30.0000,C,1.414973,1.477121


In [132]:
# data normalization, in order to train a model
df['FirstClass'] = np.where(df['Pclass'] == 1, 1, 0)
df['SecondClass'] = np.where(df['Pclass'] == 2, 1, 0)

df['EmbarkedC'] = np.where(df['Embarked'] == 'C', 1, 0)
df['EmbarkedQ'] = np.where(df['Embarked'] == 'Q', 1, 0)

df['SexNorm'] = np.where(df['Sex'] == 'female', 1, 0)

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeNorm,FareNorm,FirstClass,SecondClass,EmbarkedC,EmbarkedQ,SexNorm
0,0,3,male,22.0,1,0,7.2500,S,1.342423,0.860338,0,0,0,0,0
1,1,1,female,38.0,1,0,71.2833,C,1.579784,1.852988,1,0,1,0,1
2,1,3,female,26.0,0,0,7.9250,S,1.414973,0.898999,0,0,0,0,1
3,1,1,female,35.0,1,0,53.1000,S,1.544068,1.725095,1,0,0,0,1
4,0,3,male,35.0,0,0,8.0500,S,1.544068,0.905796,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,1.591065,1.464266,0,0,0,1,1
886,0,2,male,27.0,0,0,13.0000,S,1.431364,1.113943,0,1,0,0,0
887,1,1,female,19.0,0,0,30.0000,S,1.278754,1.477121,1,0,0,0,1
889,1,1,male,26.0,0,0,30.0000,C,1.414973,1.477121,1,0,1,0,0


In [133]:
coeff = torch.randn(8)
coeff

tensor([ 0.8656,  0.8829,  0.3426, -1.0176,  0.0419, -0.1448, -1.0782, -1.0205])

In [134]:
data = torch.tensor(df[['AgeNorm','SibSp','Parch','FareNorm','FirstClass','SecondClass','EmbarkedC','EmbarkedQ']].values)
data

tensor([[1.3424, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.5798, 1.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [1.4150, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [1.2788, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.4150, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [1.5051, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]],
       dtype=torch.float64)

In [135]:
data * coeff

tensor([[ 1.1621,  0.8829,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 1.3675,  0.8829,  0.0000,  ..., -0.0000, -1.0782, -0.0000],
        [ 1.2249,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
        ...,
        [ 1.1069,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 1.2249,  0.0000,  0.0000,  ..., -0.0000, -1.0782, -0.0000],
        [ 1.3029,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -1.0205]],
       dtype=torch.float64)